In [2]:
import numpy as np
import math

In [18]:
#keep things general at first, then we can get into specifics later

#define starting particle as parent
#define two ending particles as daughters

#input is a list of 4-momenta for the parent
#output is the 4-momenta for one of the daughters

#assuming that we know the masses of all of the particles involved (placeholders for now) + p/E of parent
mass_parent = 10
mass_daughter_1 = 6 #set as gamma for now
mass_daughter_2 = 4 #set as A' for now
momentum_parent = 20 #in lab frame, set to pion for now
E_parent = np.sqrt(mass_parent**2 + momentum_parent**2)
print("Energy of Parent: " + str(E_parent))
print("Momentum of Parent: " + str(momentum_parent))

def find_two_daughter_momenta(mass_parent, mass_daughter_1, mass_daughter_2, momentum_parent, E_parent):
    #in the COM frame, we have (1 always forwards, 2 always back):
    E_daughter_1_COM = (mass_parent**2 + mass_daughter_1 - mass_daughter_2**2)/(2*mass_parent)
    E_daughter_2_COM = mass_parent - E_daughter_1_COM
    
    p_daughter_2_COM = np.sqrt(E_daughter_2_COM**2 - mass_daughter_2**2)
    p_daughter_1_COM = -p_daughter_2_COM

    #in the lab frame, we have (1 always forwards, 2 always back):
    p_daughter_1_lab = (E_parent/mass_parent)*(p_daughter_1_COM + (momentum_parent/E_parent)*E_daughter_1_COM)
    p_daughter_2_lab = (E_parent/mass_parent)*(p_daughter_2_COM + (momentum_parent/E_parent)*E_daughter_2_COM)
    
    return p_daughter_1_lab, p_daughter_2_lab, p_daughter_1_COM, p_daughter_2_COM, E_daughter_1_COM, E_daughter_2_COM
    
print("\nwithout angles output: ")
print(find_two_daughter_momenta(mass_parent, mass_daughter_1, mass_daughter_2, momentum_parent, E_parent))

#now with angles (assuming they are specified)
theta_daughter_2 = 2 #assuming in x-direction, from 0-180 deg.(0-3.14 rad.)
phi_daughter_2 = 5 #assuming in y-direction, from 0-360 deg. (0-6.28 rad.)

def find_two_daughter_momenta_angles(theta_daughter_2, phi_daughter_2, mass_parent, mass_daughter_1, mass_daughter_2, momentum_parent, E_parent):
    #in the COM frame:
    E_daughter_1_COM = (mass_parent**2 + mass_daughter_1 - mass_daughter_2**2)/(2*mass_parent)
    E_daughter_2_COM = mass_parent - E_daughter_1_COM
    
    #find total momentum
    p_daughter_2_tot_COM = np.sqrt(E_daughter_2_COM**2 - mass_daughter_2**2)
    p_daughter_1_tot_COM = -p_daughter_2_tot_COM
    
    #set sin(2pi) = 0:
    if np.allclose(np.sin(theta_daughter_2), 0) == True:
        theta_daughter_2 = 0
    
    #find components, assuming angles are defined like spherical coord.
    p_daughter_2_x_COM = p_daughter_2_tot_COM*np.sin(theta_daughter_2)*np.cos(phi_daughter_2)
    p_daughter_2_y_COM = p_daughter_2_tot_COM*np.sin(theta_daughter_2)*np.sin(phi_daughter_2)
    p_daughter_2_z_COM = p_daughter_2_tot_COM*np.cos(theta_daughter_2)
    
    p_daughter_1_x_COM = -p_daughter_2_x_COM
    p_daughter_1_y_COM = -p_daughter_2_y_COM
    p_daughter_1_z_COM = -p_daughter_2_z_COM

    #in the lab frame, boosting only in z direction:
    p_daughter_1_z_lab = (E_parent/mass_parent)*(p_daughter_1_z_COM + (momentum_parent/E_parent)*E_daughter_1_COM)
    p_daughter_1_x_lab = p_daughter_1_x_COM
    p_daughter_1_y_lab = p_daughter_1_y_COM
    
    p_daughter_2_z_lab = (E_parent/mass_parent)*(p_daughter_2_z_COM + (momentum_parent/E_parent)*E_daughter_2_COM)
    p_daughter_2_x_lab = p_daughter_2_x_COM
    p_daughter_2_y_lab = p_daughter_2_y_COM
    
    #find total momentum now
    p_daughter_2_tot_lab = np.sqrt(p_daughter_2_z_lab**2 + p_daughter_2_x_lab**2 + p_daughter_2_y_lab**2)
    p_daughter_1_tot_lab = np.sqrt(p_daughter_1_z_lab**2 + p_daughter_1_x_lab**2 + p_daughter_1_y_lab**2)
    
    #find the new angles in the lab frame
    theta_daughter_1_lab = np.arccos(p_daughter_1_z_lab/p_daughter_1_tot_lab)
    #doesn't change b/c boosting only in z direction
    phi_daughter_1_lab = np.arctan2(p_daughter_1_y_lab, p_daughter_1_x_lab) + np.pi
    
    theta_daughter_2_lab = np.arccos(p_daughter_2_z_lab/p_daughter_2_tot_lab)
    #same as 1st b/c boosting only in z direction
    phi_daughter_2_lab = phi_daughter_1_lab
    
    #find the energy in the lab frame
    E_daughter_1_lab = (E_parent/mass_parent)*(E_daughter_1_COM - (momentum_parent/E_parent)*p_daughter_1_z_COM)
    E_daughter_2_lab = (E_parent/mass_parent)*(E_daughter_2_COM - (momentum_parent/E_parent)*p_daughter_2_z_COM)
    
    return [E_daughter_1_lab, p_daughter_1_x_lab, p_daughter_1_y_lab, p_daughter_1_z_lab], [E_daughter_2_lab, p_daughter_2_x_lab, p_daughter_2_y_lab, p_daughter_2_z_lab], theta_daughter_1_lab, phi_daughter_1_lab, theta_daughter_2_lab, phi_daughter_2_lab

test = find_two_daughter_momenta_angles(theta_daughter_2, phi_daughter_2, mass_parent, mass_daughter_1, mass_daughter_2, momentum_parent, E_parent)
print("\nwith angles output: ")
print(test)

#check energy/momentum conservation
E_cons_lab = test[0][0] + test[1][0]
p_x_cons_lab = test[0][1] + test[1][1]
p_y_cons_lab = test[0][2] + test[1][2]
p_z_cons_lab = test[0][3] + test[1][3]
print("\nEnergy Conservation Check: " + str(E_cons_lab))
print("x Momentum Conservation Check: " + str(p_x_cons_lab))
print("y Momentum Conservation Check: " + str(p_y_cons_lab))
print("z Momentum Conservation Check: " + str(p_z_cons_lab))

Energy of Parent: 22.360679774997898
Momentum of Parent: 20

without angles output: 
(0.5590284919329335, 19.44097150806707, -3.774917217635375, 3.774917217635375, 4.5, 5.5)

with angles output: 
([6.920466182056449, -0.9736768375406123, 3.291529160540758, 12.512683590466672], [15.440213592941449, 0.9736768375406123, -3.291529160540758, 7.487316409533328], 0.26773711762653274, 5.0, 0.42985452782628086, 5.0)

Energy Conservation Check: 22.360679774997898
x Momentum Conservation Check: 0.0
y Momentum Conservation Check: 0.0
z Momentum Conservation Check: 20.0
